## Submission Catboost: run average data in 1 large dataset but with loss_function=MAE
Same model as before, but the average dataset was used.
Hyperparamaters were choosen as default except for depth which was choosen to be 7 as it's a good average of the optune hyperparamaters.
Submission are done as before, but we now cap the negative values.



In [ ]:
#Create a pool of data
train_pool = Pool(X_train, y_train, cat_features=["location"])
test_pool = Pool(X_test, cat_features=["location"]) 

cat_feature = ["location"]

#init model and fit it
catboost_model = CatBoostRegressor(iterations=1000, depth=9, loss_function="MAE")
catboost_model.fit(train_pool)

In [ ]:
catboost_model.get_feature_importance() #Gives a clear indication of which features the model decides to split on

In [ ]:
#make predictions
predictions = pd.DataFrame(catboost_model.predict(test_pool))
print(predictions)
predictions = predictions.clip(lower=0)

In [ ]:
#When using non-averaged dataset the model 
catboost_model.get_feature_importance()

array([7.14454410e-01, 2.33463122e-01, 3.75397728e-01, 3.78254372e+00,
       6.35562704e+00, 6.48771233e-01, 1.91280388e-04, 5.86727196e-01,
       6.00286286e-01, 1.28453266e+00, 1.39580817e+01, 2.78000924e+00,
       2.20047230e+00, 4.79723248e+01, 2.58961845e-02, 3.21505005e-05,
       1.52380537e-01, 6.11855239e-06, 4.26232953e-03, 2.48798006e-02,
       6.49045335e-03, 4.82516561e-01, 1.26288040e-01, 2.37755133e-01,
       3.20094971e-01, 3.11507508e-01, 2.63411360e-04, 1.01815617e-02,
       9.72448560e-01, 2.07196407e-01, 3.77712014e-02, 0.00000000e+00,
       2.06891077e-03, 2.01484585e-01, 5.53600765e+00, 3.78416731e+00,
       3.62264096e-01, 1.28156413e+00, 2.02655676e+00, 3.06863983e-01,
       3.35129932e-01, 9.41018340e-01, 3.63968603e-01, 9.18614039e-04,
       1.67854391e-01, 2.77279026e-01])

## Catboost - using location per location, but without / with an avg of the X_features
Just simply running with the non averaged dataset per location. THe only changed code is below

In [ ]:
#Load inn datasets
X_test  = pd.read_parquet('data/prepared_datasets/avg/only_y_cleaned/X_test.parquet')
X_train = pd.read_parquet('data/prepared_datasets/avg/only_y_cleaned/X_train.parquet')
y_train = pd.read_parquet('data/prepared_datasets/avg/only_y_cleaned/Y_train.parquet')
y_train_a = pd.read_parquet('data/prepared_datasets/avg/only_y_cleaned/Y_train_a.parquet')
y_train_b = pd.read_parquet('data/prepared_datasets/avg/only_y_cleaned/Y_train_b.parquet')
y_train_c = pd.read_parquet('data/prepared_datasets/avg/only_y_cleaned/Y_train_c.parquet')

In [ ]:
#final training and predictions
X_train_a = X_train[X_train["location"] == "A"].drop("location", axis=1)
X_train_b = X_train[X_train["location"] == "B"].drop("location", axis=1)
X_train_c = X_train[X_train["location"] == "C"].drop("location", axis=1)

X_test_a = X_test[X_test["location"] == "A"].drop("location", axis=1)
X_test_b = X_test[X_test["location"] == "B"].drop("location", axis=1)
X_test_c = X_test[X_test["location"] == "C"].drop("location", axis=1)
          
train_pool_a = Pool(X_train_a, y_train_a)
train_pool_b = Pool(X_train_b, y_train_b)
train_pool_c = Pool(X_train_c, y_train_c)


test_pool_a = Pool(X_test_a) 
test_pool_b = Pool(X_test_b) 
test_pool_c = Pool(X_test_c) 

catboost_model_a = CatBoostRegressor(iterations=1000, loss_function="MAE")
catboost_model_b = CatBoostRegressor(iterations=1000, loss_function="MAE")
catboost_model_c = CatBoostRegressor(iterations=1000, loss_function="MAE")

catboost_model_a.fit(train_pool_a)
catboost_model_b.fit(train_pool_b)
catboost_model_c.fit(train_pool_c)


pred_a = pd.DataFrame(catboost_model_a.predict(test_pool_a))
pred_b = pd.DataFrame(catboost_model_b.predict(test_pool_b))
pred_c = pd.DataFrame(catboost_model_c.predict(test_pool_c))

predictions = pd.DataFrame(pd.concat([pred_a, pred_b, pred_c]))

In [ ]:
#Gives a clear indication of which features the model decides to split on
catboost_model_a.get_feature_importance() 
catboost_model_b.get_feature_importance()
catboost_model_c.get_feature_importance()

Which gives:

[5.62286267e-01 5.10924360e-01 6.87543964e-01 7.09947835e+00
 1.74363313e+01 9.85944986e-01 2.20760153e-04 5.58126247e-01
 4.49025269e+00 8.30970146e-01 2.18527336e+01 9.53879922e+00
 8.92806168e+00 0.00000000e+00 4.22781187e-02 3.11757422e-04
 1.56871203e-01 1.95304107e-03 2.49194089e-02 1.57744874e-02
 2.63958073e-02 3.28201853e-01 6.36442997e-01 2.39626670e-01
 3.18970195e-01 8.77317441e-01 1.17008386e-03 1.39479027e-01
 1.20973054e+00 1.72680735e-01 2.71240488e-02 0.00000000e+00
 4.68738664e-03 2.42363183e-01 4.78004233e+00 8.46048186e+00
 2.24121290e-02 9.89359658e-01 2.69676091e+00 2.51194813e+00
 5.19759356e-01 1.55562505e+00 4.77258531e-01 4.11353283e-06
 3.83763961e-02]
[1.88022554e+00 1.15419262e+00 1.00922562e+00 7.48111890e+00
 1.12733488e+01 2.72871859e+00 1.39014285e-02 1.88626231e+00
 2.97976929e+00 1.34600324e+00 1.82486388e+01 7.59204454e+00
 4.27088547e+00 0.00000000e+00 1.19532079e-02 1.10894864e-04
 2.77594143e-01 5.77725208e-03 7.84931755e-03 1.27768791e-02
 1.99097121e-02 1.06270592e+00 4.29635354e-01 5.97065686e-02
 1.14551067e+00 9.21013435e-01 1.77340617e-05 2.49663816e-02
 2.00966070e+00 9.08311203e-01 6.96748329e-02 0.00000000e+00
 7.38318777e-04 3.68993942e-01 3.11252224e+00 1.83882500e+01
 6.19693328e-02 3.92594191e+00 2.02225386e+00 4.13964400e-01
 4.39549987e-01 1.26118441e+00 9.49111657e-01 0.00000000e+00
 2.24010622e-01]
[1.05183303e+00 5.47707872e-01 4.65832167e-01 9.62827924e+00
 1.61346601e+01 5.59971031e-01 3.32030535e-06 1.83163516e+00
 1.02105691e+00 9.94807280e-01 1.55418440e+01 1.42847250e+01
 2.46223005e+00 0.00000000e+00 2.78951279e-01 2.64209261e-04
 1.41965549e+00 8.09852309e-04 3.12673556e-02 8.59573696e-03
 1.80568128e+00 8.51329761e-01 5.67418183e-01 1.13536266e-01
 1.10745070e+00 5.78633083e-01 7.94598073e-17 2.64155735e-01
 8.67329333e-01 9.04708781e-01 3.28500374e-01 0.00000000e+00
 1.06570958e-01 2.69129368e-01 1.18569463e+00 1.38882300e+01
 2.71306048e-01 6.33875372e+00 3.17699212e-01 7.24903457e-01
 7.82524613e-01 1.10959525e+00 7.16590555e-01 1.79745716e-02
 6.18155048e-01]

## Catboost but with logCosh loss function
The logCosh loss func is like the MAE but derivable and stays convex, hopefully this gives us better scores
All the code is the same, the non avg dataset is used and combined dataset is also used.
No changed is EDA

In [ ]:
#Load inn datasets
X_test  = pd.read_parquet('data/prepared_datasets/only_y_cleaned/X_test.parquet')
X_train = pd.read_parquet('data/prepared_datasets/only_y_cleaned/X_train.parquet')
y_train = pd.read_parquet('data/prepared_datasets/only_y_cleaned/Y_train.parquet')

#Create a pool of data
train_pool = Pool(X_train, y_train, cat_features=["location"])
test_pool = Pool(X_test, cat_features=["location"]) 

cat_feature = ["location"]

#init model and fit it
catboost_model = CatBoostRegressor(iterations=1000, depth=9, loss_function="LogCosh")
catboost_model.fit(train_pool)
catboost:model.get_feature_importance()

array([4.72592899e-01, 4.41204367e-01, 3.84270690e-01, 2.10027644e+00,
       6.59232391e+00, 5.12073235e-01, 4.20970309e-04, 6.54575329e-01,
       1.23954700e+00, 7.62895522e-01, 1.26428425e+01, 5.33669134e+00,
       2.00509473e+00, 4.65390852e+01, 9.04155075e-03, 0.00000000e+00,
       2.35266493e-01, 9.80361685e-05, 8.22728932e-03, 3.84898460e-02,
       9.08180785e-02, 5.32835508e-01, 1.89160001e-01, 2.02757084e-01,
       4.05391975e-01, 3.17490715e-01, 1.42871927e-06, 2.04189961e-01,
       1.34250717e+00, 2.04729946e-01, 5.97507981e-02, 0.00000000e+00,
       2.12119598e-03, 1.43597432e-01, 5.77122856e+00, 3.95295454e+00,
       4.82876996e-02, 1.23904967e+00, 1.58323040e+00, 1.14037688e+00,
       3.30905708e-01, 1.17824748e+00, 4.07264867e-01, 1.95389108e-03,
       1.79484175e-01, 4.96647498e-01])

## Improved EDA by removing duplicates 
The changed code in the EDA is below
- We tried both with only removing non-zero values and removing all duplicate values over another threshold.
- The feature importances remained virtually unchanged between tries

In [ ]:
def remove_duplicates_over_threshold(df, threshold, col):
    threshold = 4  # Set your threshold here

    # Create a boolean mask to identify consecutive duplicates (non-zero values)
    mask = df[col].ne(df[col].shift()).cumsum()

    # Create a mask to identify rows that need to be kept (consecutive zeros, first in a group, or the first row of a group)
    rank_within_group = df.groupby(mask)[col].rank(method='first')
    keep_mask = (df[col] == 0) | (rank_within_group <= threshold - 1) | (mask != mask.shift())

    # Filter the DataFrame based on the keep_mask
    df_filtered = df[keep_mask]
    return df_filtered

train_all_filtered = remove_duplicates_over_threshold(train_all, 5, "pv_measurement")
print(train_all["pv_measurement"].info())
print(train_all_filtered["pv_measurement"].info())


train_all = remove_duplicates_over_threshold(train_all, 5, "pv_measurement")

In [ ]:
def remove_duplicates_over_threshold_under_val(df, col, threshold, val):
    # Define the threshold
    threshold

    # Create a boolean mask for values less than 1
    mask = df[col] < val

    # Use cumulative sum to identify consecutive groups of values less than 1
    groups = (mask != mask.shift()).cumsum()

    # Filter out groups that don't meet the threshold
    valid_groups = groups[mask].value_counts() >= threshold
    valid_mask = groups.map(valid_groups.get).fillna(False)

    # Select rows that meet the criteria
    filtered_df = df[~valid_mask]
    return filtered_df

train_all_filtered = remove_duplicates_over_threshold_under_val(train_all, "pv_measurement", 48, 1)
print(train_all["pv_measurement"].info())
print(train_all_filtered["pv_measurement"].info())

train_all = remove_duplicates_over_threshold_under_val(train_all, "pv_measurement", 40, 1)

In [ ]:
catboost_model.get_feature_importance()

array([5.71567256e-01, 4.18943761e-01, 4.79040534e-01, 2.26873274e+00,
       6.71437767e+00, 6.56502672e-01, 1.51028125e-03, 6.30578336e-01,
       1.17645691e+00, 9.28116951e-01, 1.03661404e+01, 3.86780450e+00,
       4.41558314e+00, 2.05461327e+01, 3.07241613e-02, 2.25237295e-07,
       2.03511521e-01, 0.00000000e+00, 5.76017922e-03, 2.70136885e-02,
       3.53194734e-03, 6.25759006e-01, 1.94652305e-01, 1.73253656e-01,
       2.71579813e-01, 1.61276690e-01, 1.82465864e-04, 6.76767197e-02,
       1.15612868e+00, 2.38809134e-01, 2.74421868e-02, 0.00000000e+00,
       2.76030891e-03, 3.29892964e-01, 5.81940730e+00, 6.44946627e+00,
       5.78456790e-02, 7.20871918e-01, 1.75597218e+00, 1.10972078e+00,
       5.19280417e-01, 1.11229712e+00, 4.89601711e-01, 7.72833327e-04,
       1.72535890e-01, 2.52307844e+01])

It now weights more on the snow water feature